In [19]:
import pandas as pd
import numpy as np
import math
import data_sampling as ds
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import time

In [28]:
class naiveBayes:
    
    def __init__(self):
        self.model = None
        self.priori = None
        
    def predict(self, test):
        
        test = list(test.values)
        
        probs = []
        
        for i in range(len(test)):
        
            probs.append(self.classes_prob(self.model, test[i]))
            
        aux = None
        label = None
        prob_now = -1
        results = []
        
        for i in range(len(probs)):
            
            for key in probs[i].keys():
                
                if(label == None or probs[i][key] > aux):
                    label = key
                    aux = probs[i][key]
            
            results.append(label)   
            label = None
            aux = None
                            
        return results
    
    def classes_prob(self, means_stds, test):
                
        prob = {}
        keys = means_stds.keys()
                
        for key in keys:
            
            prob[key] = 1
            
            for i in range(len(means_stds[key])):
                                
                mean, std = means_stds[key][i][0], means_stds[key][i][1]
                
                prob[key] = prob[key] * self.gaussian_prob(test[i], mean, std)
            
            prob[key] = prob[key] * self.priori[key]
                
        return prob
    
    def score(self, true_label, pred_label):
        
        count = 0
        for i in range(len(pred_label)):
            
            if(pred_label[i] == true_label[i]):
                count = count + 1
        
        return count/len(pred_label)
                  
    #Args: pred target, DataFrame DataFrame
    #Ret: None; The means and standards deviations
    def fit(self, pred, target):
        
        attrs = pred.columns
        target = list(target.values)
        
        self.priori_class(target)
        
        classes_obj = self.separate_classes(pred, target)
        
        keys = classes_obj.keys()
        means_stds = {}
       
        for key in keys:
            means_stds[key] = []
            means_stds[key] = self.summarize(classes_obj[key], attrs)
        
        self.model = means_stds
        
    def summarize(self, pred, attrs):
         
        means_std = []
          
        for attr in attrs:
            means_std.append([np.mean(pred[attr].values) ,np.std(pred[attr].values)])
        
        return means_std
    
    def priori_class(self, target):
        
        priori = {}
        count = {}
        
        for i in set(target):
            priori[i] = 0
            count[i] = 0
        
        for i in range(len(target)):
            count[target[i]] = count[target[i]] + 1
            
        for i in priori.keys():
            priori[i] = count[i]/len(target)
            
        self.priori = priori
        
    
    def separate_classes(self, pred, target):
        
        classes = list(set(target))
        classes_obj = {}
        df = {}
        
        for i in range(len(classes)):
            classes_obj[classes[i]] = []
            df[classes[i]] = pd.DataFrame()
        
        for i in range(len(pred)):
            df[target[i]] = df[target[i]].append(pred.iloc[i])
            classes_obj[target[i]] = df[target[i]]
                        
        return classes_obj    
            
    
    def gaussian_prob(self, x, mean, stdev):
        exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
        return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent
        

In [29]:
data = pd.read_csv("balance-scale.data")
sample = ds.random_subsampling(data, 1)

pred = sample[0][0].drop(columns = "State")
target = sample[0][0]["State"]

true_label = list(sample[0][1]["State"])
test = sample[0][1].drop(columns = "State")

nb = naiveBayes()

inicio = time.time()
nb.fit(pred, target)
fim = time.time()
print("\n" + "Tempo: " + str(fim - inicio))
pred_label = nb.predict(test)
nb.score(true_label, pred_label) 


Tempo: 0.7305812835693359

Tempo: 0.7322978973388672


0.910828025477707

In [27]:
gnb = GaussianNB()
inicio = time.time()
y_pred = gnb.fit(pred, target).predict(test)
fim = time.time()
print("\n" + "Tempo: " + str(fim - inicio))

accuracy_score(true_label, y_pred)


Tempo: 0.006943464279174805


0.89171974522293